# Create and train MitoSplit-Net model

## Import required Python libraries



In [1]:
import util
import plotting
import training

import numpy as np
import matplotlib.pyplot as plt
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('axes', labelsize=20)
plt.rc('legend', fontsize=18)
from tqdm import tqdm

import tensorflow as tf

In [2]:
#Define GPU device where the code will run on
gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/')

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Create model, split dataset and train

#### No preprocessing, different batch sizes

In [4]:
base_dir = 'H:/Santi/'
data_path = base_dir+'Data/' 

#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)

#Outputs
output_data = util.load_h5(data_path, 'Proc')
print('Outputs:', output_data.shape)


Loading H:/Santi/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)

Loading H:/Santi/Data/Proc.h5
Converting to array
Outputs: (37000, 128, 128)


In [5]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=b) 

Model: ref_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
2960/2960 - 45s - loss: -1.9034e-01 - binary_accuracy: 0.9781 - val_loss: -3.6308e-01 - val_binary_accuracy: 0.9876
Epoch 2/20
2960/2960 - 45s - loss: -4.8994e-01 - binary_accuracy: 0.9820 - val_loss: -5.5039e-01 - val_binary_accuracy: 0.9827
Epoch 3/20
2960/2960 - 44s - loss: -5.5902e-01 - binary_accuracy: 0.9854 - val_loss: -5.6908e-01 - val_binary_accuracy: 0.9893
Epoch 4/20
2960/2960 - 44s - loss: -5.9835e-01 - binary_accuracy: 0.9873 - val_loss: -6.1088e-01 - val_binary_accuracy: 0.9925
Epoch 5/20
2960/2960 - 44s - loss: -6.2382e-01 - binary_accuracy: 0.9887 - val_loss: -6.1731e-01 - val_binary_accuracy: 0.9935
Epoch 6/20
2960/2960 - 44s - loss: -6.4301e-01 - binary_accuracy: 0.9896 - val_loss: -6.6040e-01 - val_binary_accuracy: 0.9902
Epoch 7/20
2960/2960 - 44s - loss: -6.5739e-01 - binary_accuracy: 0.9901 - val_loss: -6.7747e-01 - val_binary_accuracy: 0.9909
Epoch 8/20
296

In [6]:
model_path = base_dir + 'Models/'
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving H:/Santi/Models/ref_f8_c9_b8/model.h5

Saving H:/Santi/Models/ref_f8_c9_b16/model.h5

Saving H:/Santi/Models/ref_f8_c9_b32/model.h5

Saving H:/Santi/Models/ref_f8_c9_b256/model.h5
Done.

Saving H:/Santi/Models/ref_f8_c9_b8/history

Saving H:/Santi/Models/ref_f8_c9_b16/history

Saving H:/Santi/Models/ref_f8_c9_b32/history

Saving H:/Santi/Models/ref_f8_c9_b256/history
Done.

Saving H:/Santi/Models/ref_f8_c9_b8/frames_test

Saving H:/Santi/Models/ref_f8_c9_b16/frames_test

Saving H:/Santi/Models/ref_f8_c9_b32/frames_test

Saving H:/Santi/Models/ref_f8_c9_b256/frames_test
Done.


#### MitoProc, different batch sizes

In [7]:
base_dir = 'H:/Santi/'
data_path = base_dir+'Data/' 

#Inputs
input_data = util.load_h5(data_path, 'MitoProc')
print('Inputs'+':', input_data.shape)

#Outputs
output_data = util.load_h5(data_path, 'Proc')
print('Outputs:', output_data.shape)


Loading H:/Santi/Data/MitoProc.h5
Converting to array
Inputs: (37000, 128, 128)

Loading H:/Santi/Data/Proc.h5
Converting to array
Outputs: (37000, 128, 128)


In [8]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'mp_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=b) 

Model: mp_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
2960/2960 - 49s - loss: -2.1657e-01 - binary_accuracy: 0.9633 - val_loss: -4.3266e-01 - val_binary_accuracy: 0.9764
Epoch 2/20
2960/2960 - 50s - loss: -4.7917e-01 - binary_accuracy: 0.9812 - val_loss: -4.9154e-01 - val_binary_accuracy: 0.9690
Epoch 3/20
2960/2960 - 48s - loss: -5.4435e-01 - binary_accuracy: 0.9844 - val_loss: -5.8480e-01 - val_binary_accuracy: 0.9835
Epoch 4/20
2960/2960 - 47s - loss: -5.8127e-01 - binary_accuracy: 0.9861 - val_loss: -5.8878e-01 - val_binary_accuracy: 0.9854
Epoch 5/20
2960/2960 - 48s - loss: -6.0326e-01 - binary_accuracy: 0.9872 - val_loss: -6.2149e-01 - val_binary_accuracy: 0.9857
Epoch 6/20
2960/2960 - 48s - loss: -6.2469e-01 - binary_accuracy: 0.9882 - val_loss: -5.5068e-01 - val_binary_accuracy: 0.9931
Epoch 7/20
2960/2960 - 48s - loss: -6.4091e-01 - binary_accuracy: 0.9889 - val_loss: -6.6405e-01 - val_binary_accuracy: 0.9919
Epoch 8/20
2960

In [9]:
model_path = base_dir + 'Models/'
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving H:/Santi/Models/mp_f8_c9_b8/model.h5

Saving H:/Santi/Models/mp_f8_c9_b16/model.h5

Saving H:/Santi/Models/mp_f8_c9_b32/model.h5

Saving H:/Santi/Models/mp_f8_c9_b256/model.h5
Done.

Saving H:/Santi/Models/mp_f8_c9_b8/history

Saving H:/Santi/Models/mp_f8_c9_b16/history

Saving H:/Santi/Models/mp_f8_c9_b32/history

Saving H:/Santi/Models/mp_f8_c9_b256/history
Done.

Saving H:/Santi/Models/mp_f8_c9_b8/frames_test

Saving H:/Santi/Models/mp_f8_c9_b16/frames_test

Saving H:/Santi/Models/mp_f8_c9_b32/frames_test

Saving H:/Santi/Models/mp_f8_c9_b256/frames_test
Done.


#### Mito & WatProc

In [3]:
base_dir = 'H:/Santi/'
data_path = base_dir+'Data/' 

#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)


Loading H:/Santi/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)


In [4]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = 16
  
  sigmas = np.arange(0.5, 2+0.5, 0.5)
  model, history, frames_test = {}, {}, {}
  
  for s in sigmas:
    model_name = 'wp_f%i_c%i_b%i_s%.1f'%(nb_filters, firstConvSize, batch_size, s)
    print('Model:', model_name)
    #Outputs
    output_data = util.load_h5(data_path, 'WatProc_s%.1f'%s)
    print('Outputs:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 

Model: wp_f8_c9_b16_s0.5

Loading H:/Santi/Data/WatProc_s0.5.h5
Converting to array
Outputs: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 31s - loss: 0.1445 - binary_accuracy: 0.9939 - val_loss: 0.0509 - val_binary_accuracy: 0.9998
Epoch 2/20
1480/1480 - 31s - loss: 0.0352 - binary_accuracy: 0.9998 - val_loss: 0.0308 - val_binary_accuracy: 0.9998
Epoch 3/20
1480/1480 - 38s - loss: -7.3073e-02 - binary_accuracy: 0.9934 - val_loss: -9.3851e-02 - val_binary_accuracy: 0.9939
Epoch 4/20
1480/1480 - 31s - loss: -1.1502e-01 - binary_accuracy: 0.9945 - val_loss: -1.0444e-01 - val_binary_accuracy: 0.9966
Epoch 5/20
1480/1480 - 31s - loss: -1.2979e-01 - binary_accuracy: 0.9951 - val_loss: -1.2456e-01 - val_binary_accuracy: 0.9963
Epoch 6/20
1480/1480 - 31s - loss: -1.3866e-01 - binary_accuracy: 0.9954 - val_loss: -1.2963e-01 - val_binary_accuracy: 0.9960
Epoch 7/20
1480/1480 - 31s - loss: -1.4635e-01 - binary_accuracy: 0.99

In [5]:
model_path = base_dir + 'Models/'
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving H:/Santi/Models/wp_f8_c9_b16_s0.5/model.h5

Saving H:/Santi/Models/wp_f8_c9_b16_s1.0/model.h5

Saving H:/Santi/Models/wp_f8_c9_b16_s1.5/model.h5

Saving H:/Santi/Models/wp_f8_c9_b16_s2.0/model.h5
Done.

Saving H:/Santi/Models/wp_f8_c9_b16_s0.5/history

Saving H:/Santi/Models/wp_f8_c9_b16_s1.0/history

Saving H:/Santi/Models/wp_f8_c9_b16_s1.5/history

Saving H:/Santi/Models/wp_f8_c9_b16_s2.0/history
Done.

Saving H:/Santi/Models/wp_f8_c9_b16_s0.5/frames_test

Saving H:/Santi/Models/wp_f8_c9_b16_s1.0/frames_test

Saving H:/Santi/Models/wp_f8_c9_b16_s1.5/frames_test

Saving H:/Santi/Models/wp_f8_c9_b16_s2.0/frames_test
Done.


#### MitoProc & WatProc

In [6]:
base_dir = 'H:/Santi/'
data_path = base_dir+'Data/' 

#Inputs
input_data = util.load_h5(data_path, 'MitoProc')
print('Inputs'+':', input_data.shape)


Loading H:/Santi/Data/MitoProc.h5
Converting to array
Inputs: (37000, 128, 128)


In [7]:
  
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = 16
  
  sigmas = np.arange(0.5, 2+0.5, 0.5)
  model, history, frames_test = {}, {}, {}
  
  for s in sigmas:
    model_name = 'wp_mp_f%i_c%i_b%i_s%.1f'%(nb_filters, firstConvSize, batch_size, s)
    print('Model:', model_name)
    #Outputs
    output_data = util.load_h5(data_path, 'WatProc_s%.1f'%s)
    print('Outputs:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 

Model: wp_mp_f8_c9_b16_s0.5

Loading H:/Santi/Data/WatProc_s0.5.h5
Converting to array
Outputs: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 28s - loss: 0.0939 - binary_accuracy: 0.9951 - val_loss: -3.8038e-02 - val_binary_accuracy: 0.9948
Epoch 2/20
1480/1480 - 28s - loss: -7.0015e-02 - binary_accuracy: 0.9934 - val_loss: -6.6300e-02 - val_binary_accuracy: 0.9973
Epoch 3/20
1480/1480 - 28s - loss: -9.8284e-02 - binary_accuracy: 0.9945 - val_loss: -7.4795e-02 - val_binary_accuracy: 0.9895
Epoch 4/20
1480/1480 - 28s - loss: -1.1105e-01 - binary_accuracy: 0.9949 - val_loss: -1.0306e-01 - val_binary_accuracy: 0.9960
Epoch 5/20
1480/1480 - 28s - loss: -1.2061e-01 - binary_accuracy: 0.9953 - val_loss: -1.1273e-01 - val_binary_accuracy: 0.9951
Epoch 6/20
1480/1480 - 28s - loss: -1.2757e-01 - binary_accuracy: 0.9956 - val_loss: -1.1798e-01 - val_binary_accuracy: 0.9967
Epoch 7/20
1480/1480 - 28s - loss: -1.3370e-01 - bin

In [8]:
model_path = base_dir + 'Models/'
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving H:/Santi/Models/wp_mp_f8_c9_b16_s0.5/model.h5

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s1.0/model.h5

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s1.5/model.h5

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s2.0/model.h5
Done.

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s0.5/history

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s1.0/history

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s1.5/history

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s2.0/history
Done.

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s0.5/frames_test

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s1.0/frames_test

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s1.5/frames_test

Saving H:/Santi/Models/wp_mp_f8_c9_b16_s2.0/frames_test
Done.
